In [2]:
import tensorflow as tf

In [3]:
print(tf.__version__)

1.3.0


In [5]:
import os.path as osp
import sys
sys.path.insert(0,'../../')


from latent_3d_points.src.ae_templates import mlp_architecture_ala_iclr_18, default_train_params
from latent_3d_points.src.autoencoder import Configuration as Conf
from latent_3d_points.src.point_net_ae import PointNetAutoEncoder

from latent_3d_points.src.in_out import snc_category_to_synth_id, create_dir, PointCloudDataSet, \
                                        load_all_point_clouds_under_folder

from latent_3d_points.src.tf_utils import reset_tf_graph
from latent_3d_points.src.general_utils import plot_3d_point_cloud

In [6]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [7]:
top_out_dir = '../data/'          # Use to save Neural-Net check-points etc.
top_in_dir = '../data/shape_net_core_uniform_samples_2048/' # Top-dir of where point-clouds are stored.

experiment_name = 'single_class_ae'
n_pc_points = 2048                # Number of points per model.
bneck_size = 128                  # Bottleneck-AE size
ae_loss = 'chamfer'                   # Loss to optimize: 'emd' or 'chamfer'
class_name = raw_input('Give me the class name (e.g. "chair"): ').lower()

NameError: name 'raw_input' is not defined

In [ ]:
syn_id = snc_category_to_synth_id()[class_name]
class_dir = osp.join(top_in_dir , syn_id)
all_pc_data = load_all_point_clouds_under_folder(class_dir, n_threads=8, file_ending='.ply', verbose=True)

In [ ]:
train_dir = osp.join(top_out_dir, experiment_name)
load_pre_trained_ae = True
restore_epoch = 500
if load_pre_trained_ae:
    conf = Conf.load(train_dir + '/configuration')
    reset_tf_graph()
    ae = PointNetAutoEncoder(conf.experiment_name, conf)
    ae.restore_model(conf.train_dir, epoch=restore_epoch)

In [ ]:
feed_pc, feed_model_names, _ = all_pc_data.next_batch(10)
# reconstructions = ae.reconstruct(feed_pc)[0]
# latent_codes = ae.transform(feed_pc)

In [ ]:
print feed_pc.shape

In [ ]:
i = 9
plot_3d_point_cloud(feed_pc[i][:, 0], 
                    feed_pc[i][:, 1], 
                    feed_pc[i][:, 2], in_u_sphere=True);

In [ ]:
import numpy as np
new_latent_codes = ae.get_latent_codes(feed_pc)
new_decodes = ae.decode(new_latent_codes)
interp_points = ae.interpolate(np.squeeze(new_decodes[0,:,:]), np.squeeze(new_decodes[4,:,:]),10)
j = 6
plot_3d_point_cloud(interp_points[j][:, 0], 
                    interp_points[j][:, 1], 
                    interp_points[j][:, 2], in_u_sphere=True);

In [ ]:
print np.expand_dims(feed_pc[0,:,:],axis=0).shape
print interp_points[6,:,:].shape
grad = ae.gradient_of_input_wrt_loss(np.expand_dims(feed_pc[0,:,:],axis=0), np.expand_dims(interp_points[6,:,:],axis=0))

In [ ]:
print np.asarray(grad)

In [ ]:
def decoder_gradient_wrt_codes(ae_in ,in_code, gt_points):
    return ae_in.sess.run(tf.gradients(ae_in.loss, ae_in.z), feed_dict={ae_in.z: in_code, ae.gt: gt_points})
    

In [ ]:
def grad_descent(x_in, grad, gamma):
    x_out = x_in - gamma*grad
    return x_out

In [ ]:
sys.path.append('../../pointnet2/utils/')
import pc_util
import tensorflow as tf

x_in = np.expand_dims(feed_pc[0,:,:],axis=0)
x_target = np.expand_dims(interp_points[6,:,:],axis=0)
print x_target.shape
target_n_points = x_target.shape[1]

desired_n_samples = 1000

choice_idx = np.random.choice(target_n_points, desired_n_samples, replace=False)
repeat_idx = np.random.choice(target_n_points, 1, replace=False)

tmp = np.squeeze(x_target) - x_target[0,repeat_idx,:]
tmp = np.linalg.norm(tmp, axis=1)

tmp = np.where(tmp < 0.3)[0]

print tmp.shape

desired_n_samples = tmp.shape[0]

chosen_points  = np.expand_dims(x_target[0,choice_idx, :], axis=0)
repeated_points = np.expand_dims(np.repeat(x_target[0,repeat_idx,:], 2048-desired_n_samples, axis=0), axis=0)

chosen_points = np.expand_dims(x_target[0,tmp, :], axis=0)
new_target = np.hstack((chosen_points, repeated_points))
print new_target.shape
# new_target = np.vstack(x_target[1,choice_idx, :], np.repeat(x_target[1,repeat_idx,:], 2048-desired_n_samples, axis=0))

In [ ]:
points_list = []



x_in_code = ae.get_latent_codes(x_in)
# x_target_code = ae.get_latent_codes(x_target)
points_list.append(x_in_code)



new_code = x_in_code

pc_util.write_ply(np.squeeze(ae.decode(new_code)), './noisy/input0.ply')
pc_util.write_ply(np.squeeze(new_target), './noisy/gt0.ply')


print new_code

# print np.asarray(grad).shape

# grad = ae.gradient_of_input_wrt_loss(x_in, x_target)
for j in xrange(10):
    grad = decoder_gradient_wrt_codes(ae, new_code, new_target)
    grad = np.expand_dims(np.squeeze(grad),axis=0)
    newer_code = grad_descent(new_code, grad, 50)
    new_code = newer_code
    points_list.append(new_code)
    
# points_list.append(x_target_code)

# points_list.append(np.squeeze(ae.decode(new_code))
for j in xrange(11):
    pc_util.write_ply(np.squeeze(ae.decode(points_list[j])), './noisy/input'+str(j)+'.ply')
    
pc_util.write_ply(np.squeeze(new_target), './noisy/input'+str(j+1)+'.ply')